In [27]:
from tvDatafeed import TvDatafeed, Interval
import pygame
import pandas as pd
import numpy as np
from datetime import datetime
import time
pygame.init()

sound_file = "message-ringtone-magic.mp3"  # Replace with the path to your sound file
pygame.mixer.music.load(sound_file)


In [28]:
username = 'amiramirmashgh'
password = '1234Asdf'
# username, password
tv = TvDatafeed()

In [30]:
def analize_data(df_XUD):
    # Add SMA 20
    window_size = 20
    df_XUD['SMA 20'] = df_XUD['close'].rolling(window=window_size).mean()

    # ADD diff and color off the candle

    # Calculate the difference between current and previous day's closing price
    df_XUD['price_diff_CO'] = df_XUD['close'] -  df_XUD['open']

    # Add a new column to indicate green (1) or red (0)
    df_XUD['candle color'] = df_XUD['price_diff_CO'].apply(lambda x: 'green' if x > 0 else 'red')

    # Assuming your DataFrame is named df
    short_window = 12  # Short-term EMA
    long_window = 26   # Long-term EMA
    signal_window = 9  # Signal line EMA

    # Calculate Short-term EMA
    df_XUD['EMA_short'] = df_XUD['close'].ewm(span=short_window, adjust=False).mean()

    # Calculate Long-term EMA
    df_XUD['EMA_long'] = df_XUD['close'].ewm(span=long_window, adjust=False).mean()

    # Calculate MACD line
    df_XUD['MACD'] = df_XUD['EMA_short'] - df_XUD['EMA_long']

    # Calculate Signal line (Signal line is typically a 9-period EMA of MACD values)
    df_XUD['Signal_Line'] = df_XUD['MACD'].ewm(span=signal_window, adjust=False).mean()

    # Calculate MACD Histogram
    df_XUD['MACD_Histogram'] = df_XUD['MACD'] - df_XUD['Signal_Line']

    # Determine MACD trend direction (Above/Below Zero Line)
    df_XUD['MACD_Trend'] = df_XUD['MACD'].apply(lambda x: 'Above' if x > 0 else 'Below')

    # Determine Signal_Line trend direction (Above/Below Zero Line)
    df_XUD['Signal_Line_Trend'] = df_XUD['Signal_Line'].apply(lambda x: 'Above' if x > 0 else 'Below')


    # Determine Histogram color (Green if MACD > Signal Line, Red otherwise)
    df_XUD['Hist_color_V1'] = df_XUD.apply(lambda row: 'green' if row['MACD'] > row['Signal_Line'] else 'red', axis=1)


    # Function to determine color based on conditions
    def determine_color(row):
        if row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'green'  # Signal Line is below MACD and Histogram is increasing
        elif row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) < abs(row['MACD_Histogram_shifted']):
            return 'light green'  # Signal Line is above MACD and Histogram is decreasing
        elif row['MACD_Histogram'] < 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'red'  # Signal Line is above MACD and Histogram is decreasing
        else:
            return 'light red'  # Signal Line is above MACD and Histogram is not decreasing

    # Calculate the shift of the MACD_Histogram column
    df_XUD['MACD_Histogram_shifted'] = df_XUD['MACD_Histogram'].shift(1)

    # Apply the determine_color function to create the color column
    df_XUD['Hist_color_V2'] = df_XUD.apply(determine_color, axis=1)

    # Function to determine color based on conditions
    def determine_color_finisher(row):
        if (row['Hist_color_V2_shifted'] ==  'light green' or row['Hist_color_V2_shifted'] ==  'green') and row['Hist_color_V2'] == "light red":
            return 'red'  # fix first hist candle
        elif (row['Hist_color_V2_shifted'] ==  'light red' or row['Hist_color_V2_shifted'] ==  'red') and row['Hist_color_V2'] == "light green":
            return 'green'  # fix first hist candle
        else:
            return  row['Hist_color_V2'] # return defulat

    # Calculate the shift of the MACD_Histogram column
    df_XUD['Hist_color_V2_shifted'] = df_XUD['Hist_color_V2'].shift(1)

    # Advaned level: GOD bless me
    df_XUD['Hist_color_V3'] = df_XUD.apply(determine_color_finisher, axis=1)


    # Create a cross counter column
    counter = 0
    counter_list = []
    prev_color = None

    for color in df_XUD['Hist_color_V1']:
        if prev_color is None or color == prev_color:
            counter += 1
        else:
            counter = 1
        counter_list.append(counter)
        prev_color = color

    df_XUD['cross_counter'] = counter_list

    def determine_candle_from_sma20(row):
        if (row['SMA 20'] > row['low']  and row['SMA 20'] < row['close']  and row['candle color'] == 'green'):
            return 'buy' 
        elif (row['SMA 20'] > row['close']  and row['SMA 20'] < row['high']  and row['candle color'] == 'red') :
            return 'sell'  
        else:
            return  '' # return defulat

    df_XUD['inside candle (rule 1)'] = df_XUD.apply(determine_candle_from_sma20, axis=1)

    def determine_distance_sma20(row):
        if (row['inside candle (rule 1)'] == 'buy'):
            return (row['high'] - row['SMA 20'] ) 
        elif (row['inside candle (rule 1)'] == 'sell') :
            return row['SMA 20'] - row['low']
        else:
            return  'no position' # return defulat

    df_XUD['distance SMA 20 from high or close'] = df_XUD.apply(determine_distance_sma20, axis=1)


    # from distance where sma 20 get broke and the candle closed , it shouldn't bigger than 4 other candles 
    df_XUD['candle_size'] = df_XUD['high'] - df_XUD['low']
    df_XUD['candle_size_shifted1'] = df_XUD['candle_size'].shift(1)
    df_XUD['candle_size_shifted2'] = df_XUD['candle_size'].shift(2)
    df_XUD['candle_size_shifted3'] = df_XUD['candle_size'].shift(3)
    df_XUD['candle_size_shifted4'] = df_XUD['candle_size'].shift(4)

    def determine_rule2(row):
        if row['distance SMA 20 from high or close'] == 'no position':
            return "False"
        elif (row['distance SMA 20 from high or close'] < row['candle_size_shifted4'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted2'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted3'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted1']):
            return "True"  
        else:
            return  'False'

    df_XUD['rule_2'] = df_XUD.apply(determine_rule2, axis=1)  

    def determine_rule3(row):
        if (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Above') :
            return "buy"
        elif (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Below'):
            return "Half_buy"  
        elif (row['MACD_Trend'] == 'Below'  and row['Signal_Line_Trend'] == 'Below'):
            return "sell"  
        else:
            return  'Half_sell'

    df_XUD['rule_3'] = df_XUD.apply(determine_rule3, axis=1) 

    #for long sabz kamrang and for short germez kamrang?
    def determine_rule4_5(row):
        if (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light red' and row['cross_counter'] >= 4  ) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light green' and row['cross_counter'] >= 4 ) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'green' and row['cross_counter'] <= 2 ):
            return "buy"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light green' and row['cross_counter'] >= 4 ) :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light red' and row['cross_counter'] >= 4) :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'red' and row['cross_counter'] <= 2 ):
            return "sell"

        else:
            return  'unknown'

    df_XUD['rule_4_5'] = df_XUD.apply(determine_rule4_5, axis=1) 

    df_XUD['SL_sell_shifted1'] = df_XUD['high'].shift(1)
    df_XUD['SL_sell_shifted2'] = df_XUD['high'].shift(2)
    df_XUD['SL_sell_shifted3'] = df_XUD['high'].shift(3)
    df_XUD['SL_sell_shifted4'] = df_XUD['high'].shift(4)

    df_XUD['SL_buy_shifted1'] = df_XUD['low'].shift(1)
    df_XUD['SL_buy_shifted2'] = df_XUD['low'].shift(2)
    df_XUD['SL_buy_shifted3'] = df_XUD['low'].shift(3)
    df_XUD['SL_buy_shifted4'] = df_XUD['low'].shift(4)

    def determine_SL_buy(row):
       return min(row['SL_buy_shifted1'],row['SL_buy_shifted2'],row['SL_buy_shifted3'],row['SL_buy_shifted4']) - 0.1

    def determine_SL_sell(row):
       return max(row['SL_sell_shifted1'],row['SL_sell_shifted2'],row['SL_sell_shifted3'],row['SL_sell_shifted4']) + 0.1

    df_XUD['SL_sell'] = df_XUD.apply(determine_SL_sell, axis=1) 
    df_XUD['SL_buy'] = df_XUD.apply(determine_SL_buy, axis=1) 


    def determine_entry_point_buy(row):
       return row["high"] + 0.1

    def determine_entry_point_sell(row):
       return row["low"] - 0.1



    df_XUD['entry_sell'] =  df_XUD.apply(determine_entry_point_sell, axis=1) 
    df_XUD['entry_buy'] =  df_XUD.apply(determine_entry_point_buy, axis=1) 


    def determine_TP_buy(row):
       return 1.5 * (row["entry_buy"] - row["SL_buy"]) + row["entry_buy"]

    def determine_TP_sell(row):
       return 1.5 * (row["SL_sell"] -   row["entry_sell"]) + row["entry_sell"] 

    df_XUD['TP_sell'] = df_XUD.apply(determine_TP_sell, axis=1) 
    df_XUD['TP_buy'] = df_XUD.apply(determine_TP_buy, axis=1) 


    def determine_finall_desions(row):
        if (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "buy" and row["rule_4_5"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= 1 ):
            return "buy"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "sell" and row["rule_4_5"] == "sell"  and (row["SL_sell"] -   row["entry_sell"]) >= 1):
            return "sell"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "Half_sell" and row["rule_4_5"] == "sell" and (row["SL_sell"] -   row["entry_sell"]) >= 1):
            return "sell"
        elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "Half_buy" and row["rule_4_5"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= 1 ):
            return "buy"
        else :
            return ""

    df_XUD['finall_desions'] = df_XUD.apply(determine_finall_desions, axis=1) 

    # Deleting multiple columns
    columns_to_delete = ["SL_buy_shifted4","SL_buy_shifted3","SL_buy_shifted2","SL_buy_shifted1","SL_sell_shifted4","SL_sell_shifted3","SL_sell_shifted2","SL_sell_shifted1","Hist_color_V1", "MACD_Histogram_shifted", "Hist_color_V2" , "Hist_color_V2_shifted","candle_size_shifted4","candle_size_shifted3","candle_size_shifted2","candle_size_shifted1"]
    df_XUD = df_XUD.drop(columns_to_delete, axis=1)

    # print(df_XUD)
    #last candle must delete?

    # Save the DataFrame to a CSV file
    # df_XUD.to_csv('output.csv', index=True)
    df_XUD.drop(df_XUD.index[-1], inplace=False)

    # print("DataFrame saved to 'output.csv'")
    return df_XUD

In [32]:
def get_data(coin):
    # get data
    df_XUD = tv.get_hist(symbol=coin,exchange='BINANCE',interval=Interval.in_5_minute,n_bars=400)
    return df_XUD

In [1]:
list_crypto = ["GTCUSDT.P","KNCUSDT.P","HBARUSDT.P","BTCUSDT.P","MKRUSDT.P","LDOUSDT.P","KAVAUSDT.P","MATICUSDT.P","QNTUSDT.P", "FTMUSDT.P","ADAUSDT.P","BNBUSDT.P","ETHUSDT.P","APEUSDT.P","APTUSDT.P","PEOPLEUSDT.P","NEOUSDT.P","GRTUSDT.P","SNXUSDT.P","SOLUSDT.P","VETUSDT.P", "AXSUSDT.P", "XRPUSDT.P","XLMUSDT.P","ALGOUSDT.P","FILUSDT.P"] 

pygame.init()

sound_file = "message-ringtone-magic.mp3"  # Replace with the path to your sound file
pygame.mixer.music.load(sound_file)


def extract_signal(df_XUD,time):
    if df_XUD["finall_desions"][-1] == "buy" or df_XUD["finall_desions"][-1] == "buy_half" :
        pygame.mixer.music.play()
        print("time :" , time ,df_XUD["symbol"][-1] , "buy")
    elif df_XUD["finall_desions"][-1] == "sell" or df_XUD["finall_desions"][-1] == "sell_half" :
        pygame.mixer.music.play()
        print("time :" , time ,df_XUD["symbol"][-1], "sell")
    else :
        return 0   

NameError: name 'pygame' is not defined

In [34]:
def main():
    while(True):    
      current_time = datetime.now()
      minutes =int(str(current_time.minute))%5
      if (minutes == 0) and current_time.second <= 5:
         for i in range(len(list_crypto)):
            df_XUDD = analize_data(get_data(list_crypto[i]))
            extract_signal(df_XUDD,  current_time.minute )
      time.sleep(5)


if __name__ == "__main__" :
    main()

KeyboardInterrupt: 